In [ ]:
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
import re 
import json


class trafficSet(Dataset):
    def __init__(self,path,train,g_flag=False):
        with open(path,'r') as f:
              testset = json.load(f)
        actions_set = set()
        self.actions = []
        self.rewards = []
        self.data = []
        self.target = []
        for tset in testset:
            self.actions.append(tset['actions'])
            self.rewards.append(tset['robustness'][0])
            for action in tset['actions']:
                    actions_set.add(action)
        if g_flag is False:
          if train is True:
              self.data = self.actions[:int(len(self.actions)*0.8)]
              self.target = self.rewards[:int(len(self.actions)*0.8)]
          else:
              self.data = self.actions[int(len(self.actions)*0.8):]
              self.target = self.rewards[int(len(self.actions)*0.8):]
        else:
          self.data = self.actions
          self.target = self.rewards
            
        self.max_len = len(self.actions[0]) + 1
        self.actions_list = sorted(list(actions_set))
        self.actions_to_index = {self.actions_list[i] : i  for i in range(len(self.actions_list))}
        self.actions_category = []
        self.actions_dict = {}
        self.actions_index = []
        i = 0
        action_string = re.sub(r'[0-9]+', '', self.actions_list[0])
        for action in self.actions_list:
            current_action =  action[:4] + re.sub(r'[0-9]+', '', action[4:])
            if current_action not in self.actions_category:
                self.actions_category.append(current_action)
                self.actions_index.append(i)
                self.actions_dict[current_action] = [action]
            else:
                self.actions_dict[current_action].append(action)
            i = i + 1
        self.actions_indexes= {}
        for i in range(len(self.actions_index)):
            if i != len(self.actions_index) -1 :
                self.actions_indexes[self.actions_category[i]] = [self.actions_index[i],self.actions_index[i+1]]
            else:
                self.actions_indexes[self.actions_category[i]] = [self.actions_index[i],len(self.actions_list)]
            i = i + 1
        self.actions_list = self.actions_list + [',','.'] # bos,eos(.) and pad(,) tokens
        self.actions_indexes[','] = [len(self.actions_list)-2,len(self.actions_list)-1] # pad_index(,)
        self.actions_indexes['.'] = [len(self.actions_list)-1,len(self.actions_list)] # bos_index and eos_index(.)
        self.pad_index = len(self.actions_list) - 2
        self.bos_index = len(self.actions_list) - 1
        # self.embeddings = Embedding(len(self.actions_list),emb_dim,self.actions_indexes[','][0])
    def __getitem__(self, index):
        actions = self.actions[index]
        actions_idx = []
        for action in actions:
            actions_idx.append(self.actions_to_index[action])
        # action = [self.actions_indexes['.'][0]] + action  # add <BOS> for every action
        action = torch.LongTensor(actions_idx)
        reward = self.rewards[index]
        reward = np.exp(reward)
        # print(index)
        return action,reward
    def __len__(self):
        return len(self.data)

In [ ]:
model =  MLP(num_tokens=154,
                                num_outputs=1,
                                num_hid=1024,
                                num_layers=4, # TODO: add these as hyperparameters?
                                dropout=0.1,
                                max_len=38)


In [ ]:
import torch
import torch.nn as nn

def Embedding(num_embeddings, embedding_dim, padding_idx=None):
    m = nn.Embedding(num_embeddings, embedding_dim, padding_idx=padding_idx)
    nn.init.normal_(m.weight, mean=0, std=embedding_dim ** -0.5)
    if padding_idx is not None:
        nn.init.constant_(m.weight[padding_idx], 0)
    return m
    
class MLP(nn.Module):
    def __init__(self, num_tokens, num_outputs, num_hid,
                 num_layers, max_len=60, dropout=0.1,
                 partition_init=150.0,):
        super(MLP,self).__init__()
        self.input = nn.Linear(num_tokens * max_len, num_hid)
        hidden_layers = []
        for _ in range(num_layers):
            hidden_layers.append(nn.Dropout(dropout))
            hidden_layers.append(nn.ReLU())
            hidden_layers.append(nn.Linear(num_hid, num_hid))
        self.hidden = nn.Sequential(*hidden_layers)
        self.output = nn.Linear(num_hid, num_outputs)
        self.max_len = max_len
        self.num_tokens = num_tokens
        self.emb = Embedding(self.num_tokens,self.num_tokens)

   

    def forward(self, x,  return_all=False, lens=None):
        x = self.emb(x)
        # print(x.shape)
        x = x.reshape(x.size(0),-1)
        # print(x.shape)
        out = self.input(x)
        # out = x.reshape(-1)
        out = self.hidden(out)
        out = self.output(out)
        # print(out.shape)
        out = out.reshape(-1)
        return out



In [ ]:
from sklearn.metrics import mean_absolute_error
import torch
import os
import shutil
import time
import numpy as np
def save_ckpt(state, is_best, model_save_dir):
    current_w = os.path.join(model_save_dir, args.current_w)
    best_w = os.path.join(model_save_dir, args.best_w)
    torch.save(state, current_w)
    if is_best: shutil.copyfile(current_w, best_w)

def cal_mean_error(y_true,y_pred):
    y_true = y_true.view(-1).cpu().detach().numpy().astype(float)
    y_pred = y_pred.view(-1).cpu().detach().numpy().astype(float)
    return mean_absolute_error(y_true, y_pred)

def mkdirs(path):
    if not os.path.exists(path):
        os.makedirs(path)

# adjust learning rate 
def adjust_learning_rate(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
  # vocab = ["0", "1", "2", "3", "4", "5", "6", "7", "8", '9', '.', ',']    
args = AttrDict({
    "resume": False, 
    "ckpt" : 'model/', 
    "batch_size": 128,
    "model_name": "proxy",
     "stage_epoch":[32,64,128,256],
    "lr":0.01,
    "max_epoch" : 256,
    # saved current weight path
    "current_w" :'current_w.pth',
    # saved best weight path
    "best_w" : 'best_w.pth',
    "lr_decay" : 10
    
})
def print_time_cost(since):
    time_elapsed = time.time() - since
    return '{:.0f}m{:.0f}s\n'.format(time_elapsed // 60, time_elapsed % 60)

In [ ]:
import torch, time, os, shutil
import numpy as np
import pandas as pd
#from tensorboard_logger import Logger
from torch import nn, optim
from torch.utils.data import DataLoader
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(41)
torch.cuda.manual_seed(41)

def train_epoch(model, optimizer, criterion, train_dataloader, show_interval=10):
    model.train()
    print("start_training")
    mse_meter, loss_meter, it_count = 0, 0, 0
    for inputs, target in train_dataloader:
        inputs = inputs.to(device)
        target = target.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward
        output = model(inputs)
        output = output.to(torch.float32)
        target = target.to(torch.float32)
        # print(target.shape)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        loss_meter += loss.item()
        it_count += 1
        mse = cal_mean_error(target, output)
        mse_meter += mse
        if it_count != 0 and it_count % show_interval == 0:
            print("%d,loss:%.3e mse:%.3f" % (it_count, loss.item(), mse))
    return loss_meter / it_count, mse_meter / it_count


def val_epoch(model, criterion, val_dataloader, threshold=0.5):
    model.eval()
    mse_meter,loss_meter, it_count = 0, 0, 0
    with torch.no_grad():
        for inputs, target in val_dataloader:
            inputs = inputs.to(device)
            target = target.to(device)
            output = model(inputs)
            loss = criterion(output, target)
            loss_meter += loss.item()
            it_count += 1
            mse = cal_mean_error(target, output)
            mse_meter += mse
    return loss_meter / it_count, mse_meter / it_count


def train(args):
    # get model 
    model =  MLP(num_tokens=154,
                                num_outputs=1,
                                num_hid=1024,
                                num_layers=4, # TODO: add these as hyperparameters?
                                dropout=0.1,
                                max_len=38)
#     if args.ckpt and not args.resume:
#         state = torch.load(args.ckpt, map_location='cpu')
#         model.load_state_dict(state['state_dict'])
#         print('train with pretrained weight val_f1', state['f1'])
    model = model.to(device)
    print(model)
    # data
    train_dataset = trafficSet(path = "a_testset_for_double_direction.json",train=True)
    train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=6)
    val_dataset = trafficSet(path = "a_testset_for_double_direction.json",train=False)
    val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=4)
    print("train_datasize", len(train_dataset), "val_datasize", len(val_dataset))
    # optimizer and loss
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
    criterion = nn.MSELoss()
    # model save dir
    model_save_dir = '%s/%s_%s' % (args.ckpt, args.model_name, time.strftime("%Y%m%d%H%M"))
    mkdirs(model_save_dir)
    best_mse = -1
    lr = args.lr
    start_epoch = 1
    stage = 1
    # train from last save point
#     if args.resume:
#         if os.path.exists(args.ckpt):  # weight path
#             model_save_dir = args.ckpt
#             current_w = torch.load(os.path.join(args.ckpt, config.current_w))
#             best_w = torch.load(os.path.join(model_save_dir, config.best_w))
#             best_f1 = best_w['loss']
#             start_epoch = current_w['epoch'] + 1
#             lr = current_w['lr']
#             stage = current_w['stage']
#             model.load_state_dict(current_w['state_dict'])
#             if start_epoch - 1 in config.stage_epoch:
#                 stage += 1
#                 lr /= config.lr_decay
#                 utils.adjust_learning_rate(optimizer, lr)
#                 model.load_state_dict(best_w['state_dict'])
#             print("=> loaded checkpoint (epoch {})".format(start_epoch - 1))
#     logger = Logger(logdir=model_save_dir, flush_secs=2)
    # =========>start training<=========
    for epoch in range(start_epoch, args.max_epoch + 1):
        since = time.time()
        train_loss, train_mse = train_epoch(model, optimizer, criterion, train_dataloader, show_interval=100)
        val_loss, val_mse = val_epoch(model, criterion, val_dataloader)
        print('#epoch:%02d stage:%d train_loss:%.3e train_mse:%.3f  val_loss:%0.3e val_mse:%.3f time:%s\n'
              % (epoch, stage, train_loss, train_mse, val_loss, val_mse, print_time_cost(since)))
        # logger.log_value('train_loss', train_loss, step=epoch)
        # logger.log_value('train_f1', train_f1, step=epoch)
        # logger.log_value('val_loss', val_loss, step=epoch)
        # logger.log_value('val_f1', val_f1, step=epoch)
        state = {"state_dict": model.state_dict(), "epoch": epoch, "loss": val_loss, 'mse': val_mse, 'lr': lr,
                 'stage': stage}
        save_ckpt(state, best_mse > val_mse, model_save_dir)
        best_mse = max(best_mse, val_mse)
        print(best_mse)
        if epoch in args.stage_epoch:
            stage += 1
            lr /= args.lr_decay
            best_w = os.path.join(model_save_dir, args.best_w)
            model.load_state_dict(torch.load(best_w)['state_dict'])
            print("*" * 10, "step into stage%02d lr %.3ef" % (stage, lr))
            adjust_learning_rate(optimizer, lr)

train(args)

MLP(
  (input): Linear(in_features=5852, out_features=1024, bias=True)
  (hidden): Sequential(
    (0): Dropout(p=0.1, inplace=False)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
    (3): Dropout(p=0.1, inplace=False)
    (4): ReLU()
    (5): Linear(in_features=1024, out_features=1024, bias=True)
    (6): Dropout(p=0.1, inplace=False)
    (7): ReLU()
    (8): Linear(in_features=1024, out_features=1024, bias=True)
    (9): Dropout(p=0.1, inplace=False)
    (10): ReLU()
    (11): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (output): Linear(in_features=1024, out_features=1, bias=True)
  (emb): Embedding(154, 154)
)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train_datasize 3900 val_datasize 975
start_training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


#epoch:01 stage:1 train_loss:3.042e+06 train_mse:322.124  val_loss:5.327e+01 val_mse:0.458 time:0m9s


0.4575966253904035
start_training


...

#epoch:254 stage:4 train_loss:8.558e-02 train_mse:0.246  val_loss:7.174e-02 val_mse:0.228 time:0m14s


0.4575966253904035
start_training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


#epoch:255 stage:4 train_loss:2.108e-01 train_mse:0.259  val_loss:7.184e-02 val_mse:0.228 time:0m14s


0.4575966253904035
start_training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


#epoch:256 stage:4 train_loss:4.532e-01 train_mse:0.259  val_loss:7.177e-02 val_mse:0.228 time:0m14s


0.4575966253904035
********** step into stage05 lr 1.000e-06f


In [ ]:
train_dataset = trafficSet(path = "a_testset_for_double_direction.json",train=True)
train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=6)
for x,y in train_dataloader:
  print(x.shape)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([128, 38])
torch.Size([60, 38])


In [ ]:
model1 =  MLP(num_tokens=154,
                                num_outputs=1,
                                num_hid=1024,
                                num_layers=4, # TODO: add these as hyperparameters?
                                dropout=0.1,
                                max_len=38)
model1.load_state_dict(torch.load("/content/model/proxy_202206280904/current_w.pth",map_location='cpu')['state_dict'])
val_dataset = trafficSet(path = "a_testset_for_double_direction.json",train=False)
criterion = nn.MSELoss()

val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=4)

val_loss, val_mse = val_epoch(model1, criterion, val_dataloader)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
val_loss

0.07176542143914345

In [ ]:
val_mse

0.22795962596799946